In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
# Using GIBA Features
cols = ['f190486d6', '58e2e02e6', 'eeb9cd3aa', '9fd594eec', '6eef030c1', '15ace8c9f', 
        'fb0f5dbfe', '58e056e12', '20aa07010', '024c577b9', 'd6bb78916', 'b43a7cfd5', 
        '58232a6fb', '1702b5bf0', '324921c7b', '62e59a501', '2ec5b290f', '241f0f867', 
        'fb49e4212', '66ace2992', 'f74e8f13d', '5c6487af1', '963a49cdc', '26fc93eb7', 
        '1931ccfdd', '703885424', '70feb1494', '491b9ee45', '23310aa6f', 'e176a204a', 
        '6619d81fc', '1db387535', 'fc99f9426', '91f701ba2', '0572565c2', '190db8488', 
        'adb64ff71', 'c47340d97', 'c5a231d81', '0ff32eb98'
       ]


In [ ]:
import gc
gc.collect()

In [ ]:
# the below functions returns value which guarantees low RMSLE
def get_g_mean(x1):
    x = pd.to_numeric(x1)
    try:
        x = list(x1[x1>=30000])
    except:
        pass
    if len(x)>0:
        return np.exp(np.log1p(x).mean())-1
    return 1963500.0 #g_mean of target

In [ ]:
# Predicts for the matches based on the first and last n-2 features
def get_preds(data,cols_set):
    features = list(cols_set)
    d1 = data[features[:-2]].apply(tuple, axis=1).to_frame().rename(columns={0: 'key'})
    d2 = data[features[2:]].apply(tuple, axis=1).to_frame().rename(columns={0: 'key'})
    d2['pred'] = data[features[0]]
    d2 = d2[d2['pred'] != 0] 
    d3 = d2.groupby('key').agg({'pred':get_g_mean}).reset_index()
    d3.columns = ['key','pred']
    d = d1.merge(d3, how='left', on='key')
    return d.pred

In [ ]:
train = pd.read_csv('../input/train.csv')
train[:5]

In [ ]:
preds = get_preds(train,cols)

In [ ]:
def get_rmsle(actual, predicted):
    sq = np.square(np.log1p(predicted) - np.log1p(actual))
    rmsle = np.sqrt(np.mean(sq))
    return rmsle

In [ ]:
get_rmsle(train.target[pd.notna(preds)],preds[pd.notna(preds)])

# For non NAs, we get 0.8098 RMSLE on Train

In [ ]:
test = pd.read_csv('../input/test.csv')
test[:5]

In [ ]:
from tqdm import  tqdm
tqdm.pandas()

In [ ]:
test_ids = test[['ID']]
test = test.drop(columns=['ID'])

In [ ]:
test_ids['g_mean'] = test.progress_apply(lambda x: get_g_mean(x), axis=1)

In [ ]:
test = test[cols]

In [ ]:
gc.collect()

In [ ]:
test_ids['g_mean2'] = test.progress_apply(lambda x: get_g_mean(x), axis=1)

In [ ]:
gc.collect()

In [ ]:
req_cols = []
for i_ in tqdm(range(40,13,-1)):
    for i in range(40-i_+1):
        cols_set = list(cols[i:i_+1+i])
        col_name = 'lag_'+str(i_)+'_'+str(i)
        test_ids[col_name] = get_preds(test, cols_set)

In [ ]:
sol = test_ids[['ID']]

In [ ]:
req_cols = req_cols + ['g_mean','g_mean2']

In [ ]:
test_ids = test_ids[req_cols]

In [ ]:
gc.collect()

In [ ]:
def get_final(x):
    x1 = list(x[x>0])
    return (x1[0]**0.7 * get_g_mean(x1)**0.3)
sol['target'] = test_ids.progress_apply(lambda x: get_final(x),axis=1)

In [ ]:
sol.to_csv('solution_using_giba_properties.csv',index=False)